In [1]:
import json
import requests
import pandas as pd
import importlib
from requests_oauthlib import OAuth2Session
import keys

In [2]:
def loadIn():
    token=open('token.json',mode='r')
    token=token.read()
    token=json.loads(token)
    return token

In [3]:
def keychain():
    importlib.reload(keys)
    keychain = keys.keychain
    client_id = keychain['youtube']['client_id']
    scope = keychain['youtube']['scope']
    redirect = keychain['youtube']['redirect_uris'][0]
    refresh_url = keychain['youtube']['token_uri']
    session = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)
    return keychain,session,refresh_url

In [4]:
def createUrl_1(token):
    D = {}
    D['access_token'] = token['access_token']
    D['part'] = ['snippet,contentDetails,statistics']
    D['maxResults'] = '50'
    D['chart'] ='mostPopular'
    url ='https://www.googleapis.com/youtube/v3/videos'
    popular_vid = requests.get(url, params = D)
    popular_vid = popular_vid.json()
    return popular_vid,D,url

In [5]:
def getData(data,vid):
    for item in vid:
        data['videoId'].append(item['id'])
        data['channelId'].append(item['snippet']['channelId'])
        data['categoryId'].append(item['snippet']['categoryId'])
        data['channelTitle'].append(item['snippet']['channelTitle'])
        data['publishedAt'].append(item['snippet']['publishedAt'][10:12])
    return data

In [6]:
def test(token):
    data = {'videoId':[], 'channelId':[],'categoryId':[] ,'channelTitle':[], 'publishedAt':[]}
    page = []
    popular_vid,D,url=createUrl_1(token)
    for i in range(20):
        if 'nextPageToken' not in popular_vid:
            next_page = 'CAUQAA'
        else:
            next_page = popular_vid['nextPageToken']
        page.append(next_page)
        D['pageToken'] = next_page
        popular_vid = requests.get(url, params = D)
        popular_vid = popular_vid.json()
    return getData(data,popular_vid['items'])

In [7]:
def refreshToken(token):
    keychain_1,session,refresh_url=keychain()
    token = session.refresh_token(refresh_url, 
                                 client_id=keychain_1['youtube']['client_id'],
                                 client_secret=keychain_1['youtube']['client_secret'],
                                 refresh_token=token['refresh_token'])
    return token

In [8]:
def categories(token):
    url = 'https://www.googleapis.com/youtube/v3/videoCategories'
    C = {}
    C['access_token'] = token['access_token']
    C['part'] = 'snippet'
    C['regionCode'] = 'US'
    category_vid = requests.get(url, params = C)
    category_vid = category_vid.json()
    return category_vid

In [9]:
def categoryTable(json):
    data_category = {'id':[],'title':[]}
    for item in json:
        data_category['id'].append(item['id'])
        data_category['title'].append(item['snippet']['title'])
    return data_category

In [10]:
def merge(token,df):
    try:
        category_vid=categories(token)
        data_2=categoryTable(category_vid['items'])
        df1 = pd.DataFrame(data_2)
    
    except KeyError:
        new_token=refreshToken(token)

        category_vid=categories(new_token)
        data_2=categoryTable(category_vid['items'])
        df1 = pd.DataFrame(data_2)

    result = pd.merge(df, df1, left_on='categoryId', right_on='id')
    result.to_csv('result.csv')

In [11]:
def main():
    token=loadIn()
    try:
        data_1=test(token)
        
    except KeyError:
        new_token=refreshToken(token)

        data_1=test(new_token)

    df= pd.DataFrame(data_1)
    
    merge(token,df)

In [12]:
main()